In [1]:
!ls model/

glove.pt      gloveMixed.pt


In [2]:
import torch
import glove
import run
import pickle
from tools import Dictionary

In [4]:
EMBEDDING_SIZE = 128
CONTEXT_SIZE = 3
NUM_EPOCH = 100
BATHC_SIZE = 512
PATH = "model/gloveMixed.pt"

In [5]:
with open("data/f.pkl", mode='rb') as fp:
    doc = pickle.load(fp)

In [6]:
dictionary = Dictionary()
dictionary.update(doc)
# doc.vocab_size
VS = dictionary.vocab_size

In [36]:
len(list(dictionary.word2idx.keys()))

21281

In [7]:
model = glove.GloVeMixedCurvature(EMBEDDING_SIZE, CONTEXT_SIZE, VS)
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model

In [85]:
ws = torch.nn.functional.softmax(model.w)

curvatures = {
    'euc': float(model.euc.manifold.k.data.detach().numpy()),
    'hyp': float(model.hyp.manifold.k.data.detach().numpy()),
    'sph': float(model.sph.manifold.k.data.detach().numpy()),
} 

weights = {
    'euc': ws.detach().numpy()[0],
    'hyp': ws.detach().numpy()[1],
    'sph': ws.detach().numpy()[2],
} 

print(curvatures)
print(weights)

{'euc': 0.058570027351379395, 'hyp': 0.5209340453147888, 'sph': -0.28635114431381226}
{'euc': 6.9790296e-10, 'hyp': 5.0195154e-10, 'sph': 1.0}


In [22]:
emb = [
    model.euc._focal_embeddings.weight + model.euc._context_embeddings.weight,
    model.hyp._focal_embeddings.weight + model.hyp._context_embeddings.weight,
    model.sph._focal_embeddings.weight + model.sph._context_embeddings.weight,
]
emb = [e.detach().numpy() for e in emb]
emb = dict(zip(emb, ["euc", "hyp"]))

In [32]:
emb[0].shape

(21281, 128)

In [86]:
mixedGlove = {}
for w in dictionary.word2idx:
    mixedGlove[w] = {
        "euc":emb[0][dictionary.word2idx[w]],
        "hyp":emb[1][dictionary.word2idx[w]],
        "sph":emb[2][dictionary.word2idx[w]],
    }

In [91]:
embedding = {
    "curvatures": curvatures,
    "weights": weights,
    "vecs": mixedGlove,
}

In [93]:
# embedding

In [94]:
with open("mixed.pkl", "wb") as handle:
    pickle.dump(embedding, handle)